In [16]:
import os

import numpy as np
import pandas as pd

import matplotlib.pyplot as plt


In [17]:
imdb_df = pd.read_csv('top_gross_films.csv')

imdb_df.columns = ['rank', 'title', 'world_lifetime_gross', 'domestic_lifetime_gross', 'international_lifetime_gross', \
                   'studio', 'domestic_opening_gross', 'budget', 'release_date', 'MPAA_rating', 'run_time', \
                   'genre_list', 'lead_name', 'lead_birth_date', 'lead_height_m', 'lead_is_female', \
                   'audience_score', 'tomatometer']

FileNotFoundError: [Errno 2] No such file or directory: 'top_gross_films.csv'

In [10]:
imdb_df

,rank,title,world_lifetime_gross,domestic_lifetime_gross,international_lifetime_gross,studio,domestic_opening_gross,budget,release_date,MPAA_rating,run_time,genre_list,lead_name,lead_birth_date,lead_height_m,lead_is_female,audience_score,tomatometer
0,1,Avatar,2847246203,760507625.0,2086738578,Twentieth Century Fox,77025481.0,237000000.0,2009-12-16,PG-13,162.0,Action Adventure Fantasy Sci-Fi,Sam Worthington,1976-08-02,1.78,False,82.0,82.0
1,2,Avengers: Endgame,2797501328,858373000.0,1939128328,Walt Disney Studios Motion Pictures,357115007.0,356000000.0,2019-04-24,PG-13,181.0,Action Adventure Drama Sci-Fi,Robert Downey Jr.,1965-04-04,1.74,False,90.0,94.0
2,3,Titanic,2201647264,659363944.0,1542283320,Paramount Pictures,28638131.0,200000000.0,1997-12-19,PG-13,194.0,Drama Romance,Leonardo DiCaprio,1974-11-11,1.83,False,69.0,89.0
3,4,Star Wars: Episode VII - The Force Awakens,2069521700,936662225.0,1132859475,Walt Disney Studios Motion Pictures,247966675.0,245000000.0,2015-12-16,PG-13,138.0,Action Adventure Sci-Fi,Daisy Ridley,1992-04-10,1.70,True,85.0,93.0
4,5,Avengers: Infinity War,2048359754,678815482.0,1369544272,Walt Disney Studios Motion Pictures,257698183.0,NaN,2018-04-25,PG-13,149.0,Action Adventure Sci-Fi,Robert Downey Jr.,1965-04-04,1.74,False,91.0,85.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1195,"1,196",The Foreigner,145418070,34393507.0,111024563,STX Entertainment,13113024.0,35000000.0,2017-09-30,R,113.0,Action Thriller,Katie Leung,1987-08-08,1.64,True,NaN,NaN
1196,"1,197",Home on the Range,145358062,50030461.0,95327601,Walt Disney Studios Motion Pictures,13880771.0,110000000.0,2004-04-02,PG,76.0,Adventure Animation Comedy Family Music Western,Judi Dench,1934-12-09,1.55,True,29.0,53.0
1197,"1,198",The Exorcism of Emily Rose,145166804,75072454.0,70094350,Screen Gems,30054300.0,19000000.0,2005-09-09,PG-13,119.0,Drama Horror Thriller,Laura Linney,1964-02-05,1.70,True,NaN,NaN
1198,"1,199","Crazy, Stupid, Love.",145051197,84351197.0,60700000,Warner Bros.,19104303.0,50000000.0,2011-07-28,PG-13,118.0,Comedy Drama Romance,Steve Carell,1962-08-16,1.75,False,NaN,NaN


In [4]:
imdb_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 18 columns):
 #   Column                        Non-Null Count  Dtype  
---  ------                        --------------  -----  
 0   Unnamed: 0                    1000 non-null   object 
 1   title                         1000 non-null   object 
 2   world_lifetime_gross          1000 non-null   int64  
 3   domestic_lifetime_gross       981 non-null    float64
 4   international_lifetime_gross  1000 non-null   int64  
 5   studio                        973 non-null    object 
 6   domestic_opening_gross        970 non-null    float64
 7   budget                        818 non-null    float64
 8   release_date                  1000 non-null   object 
 9   MPAA_rating                   877 non-null    object 
 10  run_time                      973 non-null    float64
 11  genre_list                    1000 non-null   object 
 12  lead_name                     1000 non-null   object 
 13  lead

In [6]:
sum(imdb_df.lead_is_female)/len(imdb_df)

0.247

In [13]:
imdb_df.audience_score.where(imdb_df.tomatometer.notnull(), np.nan, inplace=True)
imdb_df.tomatometer.where(imdb_df.audience_score.notnull(), np.nan, inplace=True)

In [14]:
imdb_df

,rank,title,world_lifetime_gross,domestic_lifetime_gross,international_lifetime_gross,studio,domestic_opening_gross,budget,release_date,MPAA_rating,run_time,genre_list,lead_name,lead_birth_date,lead_height_m,lead_is_female,audience_score,tomatometer
0,1,Avatar,2847246203,760507625.0,2086738578,Twentieth Century Fox,77025481.0,237000000.0,2009-12-16,PG-13,162.0,Action Adventure Fantasy Sci-Fi,Sam Worthington,1976-08-02,1.78,False,82.0,82.0
1,2,Avengers: Endgame,2797501328,858373000.0,1939128328,Walt Disney Studios Motion Pictures,357115007.0,356000000.0,2019-04-24,PG-13,181.0,Action Adventure Drama Sci-Fi,Robert Downey Jr.,1965-04-04,1.74,False,90.0,94.0
2,3,Titanic,2201647264,659363944.0,1542283320,Paramount Pictures,28638131.0,200000000.0,1997-12-19,PG-13,194.0,Drama Romance,Leonardo DiCaprio,1974-11-11,1.83,False,69.0,89.0
3,4,Star Wars: Episode VII - The Force Awakens,2069521700,936662225.0,1132859475,Walt Disney Studios Motion Pictures,247966675.0,245000000.0,2015-12-16,PG-13,138.0,Action Adventure Sci-Fi,Daisy Ridley,1992-04-10,1.70,True,85.0,93.0
4,5,Avengers: Infinity War,2048359754,678815482.0,1369544272,Walt Disney Studios Motion Pictures,257698183.0,NaN,2018-04-25,PG-13,149.0,Action Adventure Sci-Fi,Robert Downey Jr.,1965-04-04,1.74,False,91.0,85.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1195,"1,196",The Foreigner,145418070,34393507.0,111024563,STX Entertainment,13113024.0,35000000.0,2017-09-30,R,113.0,Action Thriller,Katie Leung,1987-08-08,1.64,True,NaN,NaN
1196,"1,197",Home on the Range,145358062,50030461.0,95327601,Walt Disney Studios Motion Pictures,13880771.0,110000000.0,2004-04-02,PG,76.0,Adventure Animation Comedy Family Music Western,Judi Dench,1934-12-09,1.55,True,29.0,53.0
1197,"1,198",The Exorcism of Emily Rose,145166804,75072454.0,70094350,Screen Gems,30054300.0,19000000.0,2005-09-09,PG-13,119.0,Drama Horror Thriller,Laura Linney,1964-02-05,1.70,True,NaN,NaN
1198,"1,199","Crazy, Stupid, Love.",145051197,84351197.0,60700000,Warner Bros.,19104303.0,50000000.0,2011-07-28,PG-13,118.0,Comedy Drama Romance,Steve Carell,1962-08-16,1.75,False,NaN,NaN


In [15]:
imdb_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1200 entries, 0 to 1199
Data columns (total 18 columns):
 #   Column                        Non-Null Count  Dtype  
---  ------                        --------------  -----  
 0   rank                          1200 non-null   object 
 1   title                         1200 non-null   object 
 2   world_lifetime_gross          1200 non-null   int64  
 3   domestic_lifetime_gross       1175 non-null   float64
 4   international_lifetime_gross  1200 non-null   int64  
 5   studio                        1167 non-null   object 
 6   domestic_opening_gross        1162 non-null   float64
 7   budget                        959 non-null    float64
 8   release_date                  1200 non-null   object 
 9   MPAA_rating                   1052 non-null   object 
 10  run_time                      1168 non-null   float64
 11  genre_list                    1200 non-null   object 
 12  lead_name                     1200 non-null   object 
 13  lea